#### Final performance of PPO with perceptron as network

- Evaluated on all BBOB FIDs in 2D and 8D + BNNBO
- Comparison of schedules trained on either one FID or all BBOB FIDs via instance selection 
- Baselines: Default SMAC BlackBoxFacade + Random schedule, i.e. randomly selecting $\beta$ from $[10^{-3},10^6]$

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

from carps.analysis.gather_data import normalize_logs, get_interpolated_performance_df

In [2]:
def load_set(paths: list[str], set_id: str = "unknown") -> tuple[pd.DataFrame, pd.DataFrame]:
    logs = []
    for p in paths:
        fn = Path(p) / "trajectory.parquet"
        if not fn.is_file():
            fn = Path(p) / "runs/logs.csv"
        logs.append(pd.read_csv(fn))

    df = pd.concat(logs).reset_index(drop=True)
    df_cfg = pd.concat([pd.read_csv(Path(p) / "runs/logs_cfg.csv") for p in paths]).reset_index(drop=True)
    df["set"] = set_id
    return df, df_cfg

In [3]:
PATH = '/scratch/hpc-prf-intexml/tklenke/experiment_runs/dacboenv_ppo_semi'
NAME = 'dacboenv'

In [4]:
def create_df():
    run_dirs = [(PATH, NAME)]
    dfs = [load_set(paths=[ps], set_id=set_id) for ps, set_id in run_dirs]

    df = pd.concat([d for d, _ in dfs]).reset_index(drop=True)
    df = normalize_logs(df)

    perf = get_interpolated_performance_df(df)

    perf_time = get_interpolated_performance_df(df, x_column="time_norm")

    perf.to_parquet(f'{PATH}/perf.parquet')
    perf_time.to_parquet(f'{PATH}/perf_time.parquet')

In [5]:
create_df()

[14:32:34] INFO     Start normalization...                                                       ]8;id=43747;file:///scratch/hpc-prf-intexml/tklenke/repos/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=903551;file:///scratch/hpc-prf-intexml/tklenke/repos/CARP-S/carps/analysis/gather_data.py#602\602]8;;\

           INFO     Normalize n_trials...                                                        ]8;id=681111;file:///scratch/hpc-prf-intexml/tklenke/repos/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=883367;file:///scratch/hpc-prf-intexml/tklenke/repos/CARP-S/carps/analysis/gather_data.py#603\603]8;;\

           INFO     Normalize cost...                                                            ]8;id=918935;file:///scratch/hpc-prf-intexml/tklenke/repos/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=101719;file:///scratch/hpc-prf-intexml/tklenke/repos/CARP-S/carps/analysis/gather_data.py#605\605]8;;\

           INFO     Calc normalized incumbent cost...                                            ]8;id=822932;file:///scratch/hpc-prf-intexml/tklenke/repos/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=428885;file:///scratch/hpc-prf-intexml/tklenke/repos/CARP-S/carps/analysis/gather_data.py#621\621]8;;\

[14:32:36] INFO     Normalize time...                                                            ]8;id=753953;file:///scratch/hpc-prf-intexml/tklenke/repos/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=233123;file:///scratch/hpc-prf-intexml/tklenke/repos/CARP-S/carps/analysis/gather_data.py#641\641]8;;\

           INFO     Done.                                                                        ]8;id=103843;file:///scratch/hpc-prf-intexml/tklenke/repos/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=460989;file:///scratch/hpc-prf-intexml/tklenke/repos/CARP-S/carps/analysis/gather_data.py#644\644]8;;\

           INFO     Create dataframe for neat plotting by aligning x-axis / interpolating        ]8;id=63289;file:///scratch/hpc-prf-intexml/tklenke/repos/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=893891;file:///scratch/hpc-prf-intexml/tklenke/repos/CARP-S/carps/analysis/gather_data.py#718\718]8;;\
                    budget.                                                                                        

[14:32:41] INFO     Create dataframe for neat plotting by aligning x-axis / interpolating        ]8;id=237710;file:///scratch/hpc-prf-intexml/tklenke/repos/CARP-S/carps/analysis/gather_data.py\gather_data.py]8;;\:]8;id=291170;file:///scratch/hpc-prf-intexml/tklenke/repos/CARP-S/carps/analysis/gather_data.py#718\718]8;;\
                    budget.                                                                                        

In [6]:
# Data loading
df = pd.read_parquet(f'{PATH}/perf.parquet')

In [7]:
final = pd.DataFrame(columns=['optimizer_id', 'task_id', 'incumbent_value', 'seed'])

for problem in df['task_id'].unique():
    filtered = df[df['task_id'] == problem]

    incumbents = filtered.groupby(["optimizer_id", "seed"])["trial_value__cost_inc_norm"].min().reset_index()
    incumbents.columns = ['optimizer_id', 'seed', 'incumbent_value']
    incumbents['task_id'] = problem

    final = pd.concat([final, incumbents], ignore_index=True)

final.to_csv(f'{PATH}/final.csv')

/tmp/ipykernel_69346/2516198194.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final = pd.concat([final, incumbents], ignore_index=True)


In [8]:
final = pd.read_csv(f'{PATH}/final.csv')

In [9]:
FIGURE_PATH = '../figures'

In [10]:
df["task_id"].unique()

array(['bbob/2/1/0', 'bbob/2/10/0', 'bbob/2/11/0', 'bbob/2/12/0',
       'bbob/2/13/0', 'bbob/2/14/0', 'bbob/2/15/0', 'bbob/2/16/0',
       'bbob/2/17/0', 'bbob/2/18/0', 'bbob/2/19/0', 'bbob/2/2/0',
       'bbob/2/20/0', 'bbob/2/21/0', 'bbob/2/22/0', 'bbob/2/23/0',
       'bbob/2/24/0', 'bbob/2/3/0', 'bbob/2/4/0', 'bbob/2/5/0',
       'bbob/2/6/0', 'bbob/2/7/0', 'bbob/2/8/0', 'bbob/2/9/0',
       'bbob/8/1/0', 'bbob/8/10/0', 'bbob/8/11/0', 'bbob/8/12/0',
       'bbob/8/13/0', 'bbob/8/14/0', 'bbob/8/15/0', 'bbob/8/16/0',
       'bbob/8/17/0', 'bbob/8/18/0', 'bbob/8/19/0', 'bbob/8/2/0',
       'bbob/8/20/0', 'bbob/8/21/0', 'bbob/8/22/0', 'bbob/8/23/0',
       'bbob/8/24/0', 'bbob/8/3/0', 'bbob/8/4/0', 'bbob/8/5/0',
       'bbob/8/6/0', 'bbob/8/7/0', 'bbob/8/8/0', 'bbob/8/9/0',
       'bnnbo/BoreholeMultiFidelity', 'bnnbo/PDEVar', 'bnnbo/PestControl',
       'bnnbo/PressureVessel', 'bnnbo/SpeedReducer',
       'bnnbo/TensionCompressionString', 'bnnbo/WeldedBeamSO',
       'bnnbo/WingWeigh

In [11]:
from ioh import get_problem, ProblemClass

def get_regret(row):
    if "bbob" not in row["task_id"]:
        return row["trial_value__cost_inc"]
    splits = row["task_id"].split("/")
    d = int(splits[1])
    fid = int(splits[2])
    
    return abs(row["trial_value__cost_inc"] - get_problem(fid, 0, d, ProblemClass.BBOB).optimum.y)

df["regret"] = df.apply(get_regret, axis=1)
df["log_regret"] = df["regret"].map(lambda x: np.log(x + 1e-10))

In [12]:
final_regret = pd.DataFrame(columns=['optimizer_id', 'task_id', 'regret_value', 'seed'])

for problem in df['task_id'].unique():
    filtered = df[df['task_id'] == problem]

    incumbents = filtered.groupby(["optimizer_id", "seed"])["regret"].min().reset_index()
    incumbents.columns = ['optimizer_id', 'seed', 'regret_value']
    incumbents['task_id'] = problem

    final_regret = pd.concat([final_regret, incumbents], ignore_index=True)

/tmp/ipykernel_69346/95067566.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_regret = pd.concat([final_regret, incumbents], ignore_index=True)


In [ ]:
tasks = df["task_id"].unique()

fig, axes = plt.subplots(len(tasks), 2, figsize=(18, 4 * len(tasks)))

for i, task in enumerate(tasks):
    df_task = df[df["task_id"] == task]
    final_task = final_regret[final_regret["task_id"] == task]

    sns.lineplot(
        data=df_task[~df_task["optimizer_id"].str.contains("WEI")],
        x="n_trials_norm",
        y="regret",
        hue="optimizer_id",
        ax=axes[i, 0]
    )
    axes[i, 0].set_yscale("log")
    axes[i, 0].grid(True)
    axes[i, 0].set_title(f"Performance over time — {task}")
    axes[i, 0].get_legend().remove()

    sns.boxplot(
        data=final_task[~final_task["optimizer_id"].str.contains("WEI")],
        x="regret_value",
        y="optimizer_id",
        hue="optimizer_id",
        ax=axes[i, 1]
    )
    axes[i, 1].set_xscale("log")
    axes[i, 1].grid(True)
    axes[i, 1].set_title(f"Final Performance — {task}")

plt.tight_layout()
plt.savefig("1.png")


In [ ]:
tasks = df["task_id"].unique()

fig, axes = plt.subplots(len(tasks), 2, figsize=(18, 4 * len(tasks)))

for i, task in enumerate(tasks):
    df_task = df[df["task_id"] == task]
    final_task = final_regret[final_regret["task_id"] == task]

    sns.lineplot(
        data=df_task[~df_task["optimizer_id"].str.contains("UCB")],
        x="n_trials_norm",
        y="regret",
        hue="optimizer_id",
        ax=axes[i, 0]
    )
    axes[i, 0].set_yscale("log")
    axes[i, 0].grid(True)
    axes[i, 0].set_title(f"Performance over time — {task}")
    axes[i, 0].get_legend().remove()

    sns.boxplot(
        data=final_task[~final_task["optimizer_id"].str.contains("UCB")],
        x="regret_value",
        y="optimizer_id",
        hue="optimizer_id",
        ax=axes[i, 1]
    )
    axes[i, 1].set_xscale("log")
    axes[i, 1].grid(True)
    axes[i, 1].set_title(f"Final Performance — {task}")

plt.tight_layout()
plt.savefig("2.png")
